In [1]:
from scipy.stats import chisquare, chi2_contingency
import statsmodels.stats.multicomp
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import random
import pandas as pd
import math

# Remove homologs

In [3]:
homolog_file='/path/to/homologs.txt'
homologs={}
with open (homolog_file) as h:
    for cnt, line in enumerate(h):
        if 'Cluster' in line:
            cname=line.split(' ')[6][0:-1]
            homologs[cname]=[]
        else:
            name=line.split(' ')[1][0:-1]
            homologs[cname].append(name)
            


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/homologs.txt'

In [ ]:
def remove_homologs(file):
    with open(file) as f:
        p={}
        for cnt, line in enumerate(f):
            if line[0] == ">":
                protein=line.split(' ')[0][1:]
                for key, value in homologs.items():
                    if protein in value:
                        if key in p:
                            p[key].append(protein)
                        else:
                            p[key]=[protein]
    return p
                
                    


# Count kmers

In [ ]:
def countmultiple(file, mers):
    myhomologs=remove_homologs(file)
    d={}
    s=0
    write=True
    for mer in mers:
        with open(file) as f:
            for cnt, line in enumerate(f):
                if line[0] == ">":
                    write=True
                    protein=line.split(' ')[0][1:]
                    for key, value in myhomologs.items():
                        if protein in value and protein != key:
                            write=False
                else:
                    if write:
                        end=len(line)-1-mer
                        for i in range(0,end):
                            j=i+mer
                            key=line[i:j]
                            if key in d:
                                d[key]+=1
                            else:
                                d[key]=1
                            s+=1
    return d, s

# Define a function that runs chi2 tests with Benjamini-Hochberg multiple testing FDR correction
### One chi2 test per motif

In [ ]:
def chi2(a, b, asum, bsum, fc):
    a_motifs=[]
    b_motifs=[]
    pvals=[]
    keys=list(set(list(a.keys())+list(b.keys())))
    for key in keys:
        if key in a:
            apos=a[key]
        else:
            apos=0.0
        aneg=asum-apos
        if key in b:
            bpos=b[key]
        else:
            bpos=0.0
        bneg=bsum-bpos
        if apos !=0.0 and bpos!=0.0:
            table=[[apos,aneg],[bpos,bneg]]
            #print(table)
            stat, p, dof, expected = chi2_contingency(table)
            pvals.append([key,p])
            #add bh here
    pv=list(map(list, zip(*pvals)))[1]
    sq=statsmodels.stats.multitest.multipletests(pv, method="fdr_bh")
    i=0
    for reject in sq[0]:
        key=pvals[i][0]
        if key in a:
            apos=a[key]
        else:
            apos=0
        aneg=asum-apos
        if key in b:
            bpos=b[key]
        else:
            bpos=0
        bneg=bsum-bpos
        if reject:
            amore=np.log2(apos/asum)-np.log2(bpos/bsum)
            bmore=np.log2(bpos/bsum)-np.log2(apos/asum)
            #print("amore: ",amore,"   bmore: ",bmore)
            if amore > fc: 
                a_motifs.append([key,sq[1][i],amore])
            elif bmore > fc: 
                b_motifs.append([key,sq[1][i], bmore])
                #motif, adjusted p-value, fold change
        i+=1
    return a_motifs, b_motifs

# Run

In [ ]:
mers=[1,2,3,4]
path='/path/to/protein_fastas/'


s5d, s5sum=countmultiple(path+'seawater5.fa',mers)
s10d, s10sum=countmultiple(path+'seawater10.fa',mers)
b5d, b5sum=countmultiple(path+'brine5.fa',mers)
b10d, b10sum=countmultiple(path+'brine10.fa',mers)
t5d, t5sum=countmultiple(path+'all5.fa',mers)
t10d, t10sum=countmultiple(path+'all10.fa',mers)
#dbd, dbsum=countmultiple(path+'database_nonewline.fa',mers)



nots5d, nots5sum=countmultiple(path+'not_seawater5.fa',mers)
nots10d, nots10sum=countmultiple(path+'not_seawater10.fa',mers)
notb5d, notb5sum=countmultiple(path+'not_brine5.fa',mers)
notb10d, notb10sum=countmultiple(path+'not_brine10.fa',mers)
nott5d, nott5sum=countmultiple(path+'not_all5.fa',mers)
nott10d, nott10sum=countmultiple(path+'not_all10.fa',mers)

fc=0.5
s5n, ns5=chi2(s5d,nots5d,s5sum,nots5sum,fc)
s10n, ns10=chi2(s10d,nots10d,s10sum,nots10sum,fc)
b5n, nb5=chi2(b5d,notb5d,b5sum,notb5sum,fc)
b10n, nb10=chi2(b10d,notb10d,b10sum,notb10sum,fc)
t5n, nt5=chi2(t5d,nott5d,t5sum,nott5sum,fc)
t10n, nt10=chi2(t10d,nott10d,t10sum,nott10sum,fc)

In [ ]:
print("TEMPERATURE")
print('Motifs significant increase in -5C vs. not -5C proteins: ',len(t5n))
print('Motifs significant increase in not -5C vs. -5C proteins: ',len(nt5))
print('Motifs significant increase in -10C vs. not -10C proteins: ',len(t10n))
print('Motifs significant increase in not -10C vs. -10C proteins: ',len(nt10))
print("")
print("SALINITY")
print('Motifs significant increase in -5C+ASW vs. not -5C+ASW proteins: ',len(s5n))
print('Motifs significant increase in not -5C_ASW vs. -5C+ASW proteins: ',len(ns5))
print('Motifs significant increase in -5C+ASW vs. not -5C+brine proteins: ',len(b5n))
print('Motifs significant increase in not -5C_ASW vs. -5C+brine proteins: ',len(nb5))
print("")
print('Motifs significant increase in -10C+ASW vs. not -10C+ASW proteins: ',len(s10n))
print('Motifs significant increase in not -10C+ASW vs. -10C+ASW proteins: ',len(ns10))
print('Motifs significant increase in -10C+brine vs. not -10C+brine proteins: ',len(b10n))
print('Motifs significant increase in not -10C+brine vs. -10C+brine proteins: ',len(nb10))



In [ ]:
# Plot amino acid composition of mers, regardless of index

In [ ]:

def kmer_counts_no_index(kmers):
    #d[size][index][character]:count
    d={}
    for kmer in kmers:
        length=len(kmer)
        if length not in d:
            d[length]={}
        for i,k in enumerate(kmer):
#             if i not in d[length]:
#                 d[length][i]={}
            if k not in d[length]:
                d[length][k] = 0
            d[length][k] += 1
    return d
            
    
    
s5counts_no_index=kmer_counts_no_index(list(map(list, zip(*s5n)))[0])
b5counts_no_index=kmer_counts_no_index(list(map(list, zip(*b5n)))[0])
t5counts_no_index=kmer_counts_no_index(list(map(list, zip(*t5n)))[0])
t10counts_no_index=kmer_counts_no_index(list(map(list, zip(*t10n)))[0])

In [ ]:
def barchart(blue,orange,number,blue_label,orange_label):
    ind={k: v for k, v in sorted(blue[number].items(), key=lambda item: item[1], reverse=True)}.keys()
    ind=list(ind)+list(set(orange[number].keys())-set(ind))
    indnums=np.arange(0,len(ind))

    blue_ind=[blue[number].get(key) for key in ind]
    orange_ind=[orange[number].get(key) for key in ind]

    blue_ind = [0 if x is None else x for x in blue_ind]
    orange_ind = [0 if x is None else x for x in orange_ind]

    blue_ind = [x / sum(blue[number].values()) for x in blue_ind]
    orange_ind = [x  / sum(orange[number].values()) for x in orange_ind]
    width = 0.35

    plt.figure(figsize=(12,8))
    plt.bar(indnums - width/2,blue_ind, width)
    plt.bar(indnums + width/2,orange_ind, width)
    plt.xticks(indnums,ind)
    plt.ylabel('Frequency', fontsize=18)
    plt.ylabel('Amino Acid', fontsize=18)
    plt.title('Amino acid frequency among '+str(number)+'-mer polypeptides', fontsize=18)
    plt.legend([blue_label,orange_label], fontsize=14)


In [ ]:
barchart(t10counts_no_index,t5counts_no_index,4,'-10C','-5C')

In [ ]:
barchart(t10counts_no_index,t5counts_no_index,3,'-10C','-5C')

In [ ]:
barchart(b5counts_no_index,s5counts_no_index,4,'Brine','ASW')

In [ ]:
barchart(b5counts_no_index,s5counts_no_index,3,'Brine','ASW')